In [1]:
from utils import configuration, training_utilities
from model.deep_one_class import DeepOneClass
import builtins

ModuleNotFoundError: No module named 'BaseAgent'

In [ ]:
model_parameters = configuration.get_config_from_json(description="model parameters", config_file="configs/model_parameters.json")

In [ ]:
model = DeepOneClass(configuration.get_config_from_json(description="Model Parameters", config_file="configs/model_parameters.json"))

In [ ]:
builtins.training_config = configuration.get_config_from_json(description="Training Model Parameters", config_file="configs/training_parameters.json")
builtins.model_config = configuration.get_config_from_json(description="Model Configuration", config_file="configs/model_config.json")

In [ ]:
# data = loading_dataset.load_dataset()

In [ ]:
training_utilities.set_optimization(model)

In [ ]:
model_config = {
    'PRE-TRAINED_MODEL_FLAG': False,
    'SAVE_PATH': "../models/",
    'LOAD_MODEL': "0.0014488688902929425.pt",
}

model_params = {
        'CONV_LAYERS': 5,
        'INPUT_CHANNELS': [1, 30, 30, 40, 50],
        'LEFT_PAD': [4, 3, 2, 2, 2],
        'RIGHT_PAD': [5, 4, 3, 2, 2],
        'OUTPUT_CHANNELS': [30, 30, 40, 50, 50],
        'KERNEL': [10, 8, 6, 5, 5],
        'STRIDE': 1,
        'PADDING': 0,
        'SEQUENCE_LENGTH': 599
}

dataset_config = {
        'DATA_PATH': "../data/", 
}